In [1]:
DATE = "0823_stronger_aug"
DATA_TYPE = 'new_normal' # 'abn1' or 'abn2' or 'normal'
MODEL_EPOCH = '_25ep' ###### 아래 스냅샷 파일과 일치하는지 반드시 확인!!!!!!! ######

SCORE_VALUE = 0.072
#PRINT_BENIGN_DETECTION = False

import os
import pandas as pd

result_save_base_path = '/home/huray/workspace/han_work/han_test/results/'
#result_save_path = os.path.join(result_save_base_path, DATE, DATA_TO_TEST + MODEL_EPOCH)
result_save_path = os.path.join(result_save_base_path, DATE + MODEL_EPOCH, DATA_TYPE)
if not os.path.exists(result_save_path):
    os.makedirs(result_save_path)

In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import keras
import keras.preprocessing.image
from keras_retinanet.models.resnet import custom_objects
# from keras_retinanet.preprocessing.coco import CocoGenerator
from keras_retinanet.preprocessing.csv_generator import CSVGenerator

import matplotlib.pyplot as plt
import cv2

import numpy as np
import time

import tensorflow as tf

def get_session():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    return tf.Session(config=config)

normal_img_path = []
normal_x1 = []
normal_y1 = []
normal_x2 = []
normal_y2 = []
normal_class_name = []

# os.environ["CUDA_VISIBLE_DEVICES"] = "0"
keras.backend.tensorflow_backend.set_session(get_session())

/home/huray/venvs/ml/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Load RetinaNet model

In [3]:
model = keras.models.load_model('snapshots/0620_stronger_aug/resnet101_csv_25.h5', custom_objects=custom_objects)

/home/huray/venvs/ml/lib/python3.5/site-packages/keras/models.py:274: UserWarning: Output "nms" missing from loss dictionary. We assume this was done on purpose, and we will not be expecting any data to be passed to "nms" during training.
  sample_weight_mode=sample_weight_mode)


## Initialize data generators

In [4]:
# create image data generator object
val_image_data_generator = keras.preprocessing.image.ImageDataGenerator()

if DATA_TYPE == 'abn1':
    csv_path = '/home/huray/data/NCC/img_retinanet/data_abn1.csv'
elif DATA_TYPE == 'abn2':
    csv_path = '/home/huray/data/NCC/img_retinanet/data_abn2.csv'
elif DATA_TYPE == 'new_normal':
    csv_path = '/home/huray/workspace/han_work/han_test/data_normal_merged.csv'
else:
    print('WRONG DATA CSV PATH!')
    raise

# create a generator for testing data
val_generator = CSVGenerator(
    csv_path,
    '/home/huray/data/NCC/img_retinanet/class_2_onlyM.csv',
    val_image_data_generator,
    batch_size=1,
    is_testing=True,
)

test_file_df = pd.read_csv(csv_path, header=None)
num_of_test_files = len(test_file_df)
print(num_of_test_files)

1671


## Run detection

In [5]:
img_index_list = list(range(0, num_of_test_files))
img_index_list_temp = img_index_list.copy()
print(len(img_index_list))

1671


In [6]:
print("Make normal training data with annotation from normal dicom file.")
print("Start!")
print("Processing.....wait......")
# process time
start = time.time()

total_number_of_detected_annotation = 0
for index in img_index_list:
    pass_this_file = False
    
    # load image
    image, image_path = val_generator.load_image(index, get_image_path=True)
    #print(image_path)

    # copy to draw on
    draw = image.copy()
    draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

    # preprocess image for network
    image = val_generator.preprocess_image(image)
    image, scale = val_generator.resize_image(image)
    
    _, _, detections = model.predict_on_batch(np.expand_dims(image, axis=0))

    # compute predicted labels and scores
    predicted_labels = np.argmax(detections[0, :, 4:], axis=1)
    scores = detections[0, np.arange(detections.shape[1]), 4 + predicted_labels]

    # correct for image scale
    detections[0, :, :4] /= scale
      
    # 이미지 하나에 annotation 여러개 있는 경우 있으므로 리스트로 저장해 둠.
    x1_tmp = []
    y1_tmp = []
    x2_tmp = []
    y2_tmp = []
    cls_tmp = []
    
    # save detected object annotations
    number_of_detected_annotation = 0

    for idx, (label, score) in enumerate(zip(predicted_labels, scores)):
        if score < SCORE_VALUE:
           continue

        b = detections[0, idx, :4].astype(int)

        # 검출된 좌표가 minus인 경우 0으로 변경
        b[b<0] = 0
        
        x1_tmp.append(b[0])
        y1_tmp.append(b[1])
        x2_tmp.append(b[2])
        y2_tmp.append(b[3])
        cls_tmp.append('N')

        number_of_detected_annotation += 1
    
        # 이미지에 detect된 innotation 표시
        #cv2.rectangle(draw, (b[0], b[1]), (b[2], b[3]), (0, 255, 0), 5)

    for i_index in range(number_of_detected_annotation):
        result_image_path = os.path.join(result_save_path, image_path.split('/')[-1].split('.')[0]+'_'+str(i_index)+'.jpg')
        cv2.imwrite(result_image_path, draw)
        
        #print("image:%s" % result_image_path)
        #print("annotations:%d,%d,%d,%d" %(x1_tmp[i_index],y1_tmp[i_index],x2_tmp[i_index],y2_tmp[i_index]))

        normal_img_path.append(result_image_path)
        normal_x1.append(x1_tmp[i_index])
        normal_y1.append(y1_tmp[i_index])
        normal_x2.append(x2_tmp[i_index])
        normal_y2.append(y2_tmp[i_index])
        normal_class_name.append(cls_tmp[i_index])

    total_number_of_detected_annotation += number_of_detected_annotation        

print("End.")    
    
print("total detected annotation number : %d" % total_number_of_detected_annotation)
print("Process Runtime : %0.2f Minutes"%((time.time() - start)/60))

Make normal training data with annotation from normal dicom file.
Start!
Processing.....wait......
End.
total detected annotation number : 4364
Process Runtime : 11.05 Minutes


## Create CSV

In [7]:
normal_data_df = pd.DataFrame({'img_path':normal_img_path, 'x1':normal_x1, 'y1':normal_y1, 'x2':normal_x2, 'y2':normal_y2, 'class_name':normal_class_name})
normal_data_df = normal_data_df[['img_path', 'x1', 'y1', 'x2', 'y2', 'class_name']]

In [8]:
normal_data_df.to_csv('/home/huray/workspace/han_work/han_test/data_normal_annotation.csv', header=False, index=False)

In [9]:
print("DONE!")

DONE!
